In [ ]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

from scipy.stats import kendalltau as tau
from sklearn.metrics import r2_score as r2
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

from search_spaces import load_proxylessnas
from neural_predictor import regressor
from input_preprocessing import preprocess_proxylessnas
from random_search import proxylessnas_random

In [16]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

## ImageNet Experiments

In [17]:
N = 119
K = 137

In [23]:
ds = tfds.load('yelp_polarity_reviews', as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/560000 [00:00<?, ? examples/s]

Shuffling yelp_polarity_reviews-train.tfrecord...:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test examples...:   0%|          | 0/38000 [00:00<?, ? examples/s]

Shuffling yelp_polarity_reviews-test.tfrecord...:   0%|          | 0/38000 [00:00<?, ? examples/s]

Dataset yelp_polarity_reviews downloaded and prepared to /data2/home/patara/tensorflow_datasets/yelp_polarity_reviews/0.2.0. Subsequent calls will reuse this data.


### Create (architecture, validation accuracy) paris

In [9]:
models = proxylessnas_random(N + 100 + 100000)

In [10]:
train_models = models[:N]
valid_models = models[N:N+100]
test_models = models[N+100:N+100+100000]

In [ ]:
val_acc = []
test_acc = []


for m in train_models:
    tf.keras.backend.clear_session()
    model = load_proxylessnas()(weights=None, arch_kernel=m)
    
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(0.001, decay_steps=10000)
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr_schedule, rho=0.9, momentum=0.9, epsilon=0.1))
    model.fit(train, epochs=90, batch_size=128)
    loss, accuracy = model.evaluate(test)
    if accuracy > best_accuracy: 
        best_model = model

### Training and Validating the Neural Predictor

In [ ]:
train_data = 
reg = regressor(train_data, labels, N=N, n_gcn=18, n_hidden_fc=[512, 128], init_lr=0.001, weight_decay=0.00001)

### Predictive Performance Analysis of the Predictor

### Finding High Quality Mobile Sized Models